# **`Library`**

In [103]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [104]:
plt.style.use('ggplot')
plt.style.use('fivethirtyeight')

# **`Data`**

In [105]:
data = pd.read_csv('hourlyCalories_merged.csv')

In [106]:
#Raw Data
data

,Id,ActivityHour,Calories
0,1503960366,4/12/2016 12:00:00 AM,81
1,1503960366,4/12/2016 1:00:00 AM,61
2,1503960366,4/12/2016 2:00:00 AM,59
3,1503960366,4/12/2016 3:00:00 AM,47
4,1503960366,4/12/2016 4:00:00 AM,48
...,...,...,...
22094,8877689391,5/12/2016 10:00:00 AM,126
22095,8877689391,5/12/2016 11:00:00 AM,192
22096,8877689391,5/12/2016 12:00:00 PM,321
22097,8877689391,5/12/2016 1:00:00 PM,101


# **`Date/Time Format`**

In [107]:
#Convert 12 hr to 24 hr and datetime type 
date_inx = pd.to_datetime(data['ActivityHour'], format='%m/%d/%Y %I:%M:%S %p')

In [108]:
#New Dataframe with 24 hr format and Date as index for modeling 
data_m1 = pd.concat([date_inx, data['Calories']], axis=1)

In [120]:
data_m1

,ActivityHour,Calories
0,2016-04-12 00:00:00,81
1,2016-04-12 01:00:00,61
2,2016-04-12 02:00:00,59
3,2016-04-12 03:00:00,47
4,2016-04-12 04:00:00,48
...,...,...
22094,2016-05-12 10:00:00,126
22095,2016-05-12 11:00:00,192
22096,2016-05-12 12:00:00,321
22097,2016-05-12 13:00:00,101


In [110]:
#Split the time from date and set time as index 
d_m2 = pd.concat([data_m1['ActivityHour'].astype(str).str.split(' ').str[1], data['Calories']],  axis=1).set_index('ActivityHour')

In [111]:
d_m2

,Calories
ActivityHour,
00:00:00,81
01:00:00,61
02:00:00,59
03:00:00,47
04:00:00,48
...,...
10:00:00,126
11:00:00,192
12:00:00,321


In [112]:
#Group by time and agg median
d_m3 = d_m2.groupby(d_m2.index).median().reset_index()

In [113]:
d_m3

,ActivityHour,Calories
0,00:00:00,69.0
1,01:00:00,68.0
2,02:00:00,68.0
3,03:00:00,65.0
4,04:00:00,65.0
5,05:00:00,68.0
6,06:00:00,73.0
7,07:00:00,83.0
8,08:00:00,84.0
9,09:00:00,86.0


# **`Visuals`**

In [114]:
import plotly.express as px
fig = px.bar(d_m3, x='ActivityHour', y='Calories', template='plotly_dark', color= 'Calories', title='MAX VALUE: 18') 
fig.show()

In [115]:
import plotly.graph_objects as go

fig = go.Figure(go.Barpolar(
    r = d_m3.Calories,
    theta = d_m3.ActivityHour,
    width=[1],
    marker_color= 'wheat',
    marker_line_color="black",
    marker_line_width=1,
    opacity=0.8,
    hoverinfo='''r,theta''',
    name='Calories Burned',
))

fig.add_trace(go.Barpolar(
    r= d_m3[d_m3['Calories'] == d_m3['Calories'].max()]['Calories'],
    name='MAX', 
    theta = d_m3[d_m3['Calories'] == d_m3['Calories'].max()]['ActivityHour'],
    marker_color='rgb(106,81,163)',
))

fig.update_layout(
    template='plotly_dark',
    polar = 
    dict(
        radialaxis = dict(range=[0, max(d_m3['Calories'] + 2)], showticklabels=True, ticks=''),
        angularaxis = dict(showticklabels=True, ticks='',  rotation=90, direction="clockwise"),
    ),
    autosize=False,
    width=850,
    height=850,
    title = 'Median Calories Burned (military hours)'
)


fig.show()

In [116]:
# 24 hr to 12 hr conversion 
time_convert = datetime.strptime("08:00:00", "%H:%M:%S")
time_convert.strftime("%I:%M:%S %p")

'08:00:00 AM'

# **`Multiple Max/Agg Values`**

In [117]:
#Test to see if having 2 max value ex.(103,103) will transfer to the graph 
d_m3.at[13,'Calories']= 103.0

In [118]:
import plotly.express as px
fig = px.bar(d_m3, x='ActivityHour', y='Calories', template='plotly_dark', color= 'Calories', title='MAX VALUE: 13,18') 
fig.show()

In [119]:
import plotly.graph_objects as go

fig = go.Figure(go.Barpolar(
    r = d_m3.Calories,
    theta = d_m3.ActivityHour,
    width=[1],
    marker_color= 'wheat',
    marker_line_color="black",
    marker_line_width=1,
    opacity=0.8,
    hoverinfo='''r,theta''',
    name='Calories Burned',
))

fig.add_trace(go.Barpolar(
    r= d_m3[d_m3['Calories'] == d_m3['Calories'].max()]['Calories'],
    name='MAX', 
    theta = d_m3[d_m3['Calories'] == d_m3['Calories'].max()]['ActivityHour'],
    marker_color='rgb(106,81,163)',
))

fig.update_layout(
    #Change Template to remove dark mode
    template='plotly_dark',
    polar = 
    dict(
        radialaxis = dict(range=[0, max(d_m3['Calories'] + 2)], showticklabels=True, ticks=''),
        angularaxis = dict(showticklabels=True, ticks='',  rotation=90, direction="clockwise"),
    ),
    autosize=False,
    width=850,
    height=850,
    title = 'Median Calories Burned (military hours) w/Two max'
)


fig.show()